# Notebook for identifying and removing bottlenecks from ICET implementation

In [19]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time

# #limit GPU memory ------------------------------------------------
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#   try:
#     memlim = 4*1024
#     tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
#   except RuntimeError as e:
#     print(e)
# #-----------------------------------------------------------------
tf.config.set_visible_devices([], 'GPU') #run on CPU only -- seems to actually execute main parts of code faster here...

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas

%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

# %%bash
# # python -m cProfile scan_match.py
# python scan_match.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [20]:
basepath = '/media/derm/06EF-127D1/KITTI'
# sequence = '03' #forest
sequence = '09' #trees and small town
dataset = pykitti.odometry(basepath, sequence)
velo1 = dataset.get_velo(400)
c1 = velo1[:,:3]
velo2 = dataset.get_velo(401)
c2 = velo2[:,:3]

it = ICET(cloud1 = c1, cloud2 = c2, fid = 70, niter = 9, 
           draw = False, group = 2, RM = True, DNN_filter = True)

Ground truth poses are not avaialble for sequence 09.

 loading model took 0.7905905246734619 
 total:  0.790595531463623

 converting to spherical took 0.0188748836517334 
 total:  0.8095214366912842

 took  0.14232754707336426 seconds to get points in cluster

 getting spherical grid 1.065835952758789 
 total:  1.8753769397735596

 took  0.014351129531860352 seconds to get points in cluster

 took  0.010328292846679688 seconds to get points in cluster

 took  0.009932994842529297 seconds to get points in cluster

 fit_gaussian for scan 1 0.18608570098876953 
 total:  2.061474561691284

 took  0.14179468154907227 seconds to get points in cluster

 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.29549551010131836 
 total:  2.3571794033050537 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor([ 0.32096633  0.01419681  0.00261898 -0.00059351  0.00140323 -0.00501048], shape=(6,), dtype=float32)

 ~~~~~~~~~~~~~~ 
 correcting solution estimate 0.011981964111328125 
 total:  2.3691823482

# fit_gaussian()

In [178]:
def fg2(cloud, rag, npts):
    """new method of fitting gaussian to better handle ragged input data"""
    numSamples = 3
    
    coords = tf.gather(cloud, rag)
    mu = tf.math.reduce_mean(coords, axis = 1)[:,None]
#     mu = tf.math.reduce_mean(coords, axis = 1) #old
#     print(mu)

#   TODO: try randomly sampling 30 points from each ragged cell, use reduced num pts to calculate covariance
#     subsampled = tf.map_fn(sample, it.inside2) #works but SLOW
#     subsampled = tf.map_fn(sample, it.inside2, parallel_iterations=True)
#     subsampled = tf.gather(rag,tf.range(tf.shape(rag)[0]))[:numSamples] #wrong
#     print(subsampled)

    xpos = tf.gather(cloud[:,0], rag)
    ypos = tf.gather(cloud[:,1], rag)
    zpos = tf.gather(cloud[:,2], rag)
#     c = tfp.stats.covariance(xpos.to_tensor(), ypos.to_tensor())

#     print(xpos)
    idx = tf.range(30)
    xpos = tf.gather(xpos, idx, axis = 1)
    ypos = tf.gather(ypos, idx, axis = 1)
    zpos = tf.gather(zpos, idx, axis = 1)
    print(xpos)

    xx = tf.math.reduce_sum(tf.math.square(xpos - mu[:,:,0] ), axis = 1)/npts
    yy = tf.math.reduce_sum(tf.math.square(ypos - mu[:,:,1] ), axis = 1)/npts
    zz = tf.math.reduce_sum(tf.math.square(zpos - mu[:,:,2] ), axis = 1)/npts
    xy = tf.math.reduce_sum( (xpos - mu[:,:,0])*(ypos - mu[:,:,1]), axis = 1)/npts  #+
    xz = tf.math.reduce_sum( (xpos - mu[:,:,0])*(zpos - mu[:,:,2]), axis = 1)/npts #-
    yz = tf.math.reduce_sum( (ypos - mu[:,:,1])*(zpos - mu[:,:,2]), axis = 1)/npts #-

    sigma = tf.Variable([xx, xy, xz,
                        xy, yy, yz,
                        xz, yz, zz]) 
    sigma = tf.reshape(tf.transpose(sigma), (tf.shape(sigma)[1] ,3,3))
        
#     mu = None
    return(mu, sigma)

@tf.function
def sample(x, samples=3):
  """https://stackoverflow.com/questions/71073873/sample-from-ragged-tensor"""  
  length = tf.shape(x)[0]
#   was this
#   x = tf.cond(tf.less_equal(length, samples), lambda: x, lambda: tf.gather(x, tf.random.shuffle(tf.range(length))[:samples]))
 
#   test
#   x = tf.cond(tf.less_equal(length, samples), lambda: x, lambda: tf.gather(x, tf.range(length))[:samples])
  x = tf.gather(x,tf.range(length))[:samples]

    
  return x

In [179]:
s = time.time()
mu2, sigma2 = it.fit_gaussian(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print("\n took", time.time() - s, " s with old method")

s = time.time()
mu2, sigma2 = fg2(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print(" \n took", time.time() - s, " s with new method")

# print(it.npts2)
# print(it.inside2)

tf.Tensor(
[[ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 [ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 [ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 ...
 [-1.9604998  -1.9519119  -1.9373393  ... -1.4723451  -1.4607754
  -1.4521916 ]
 [-1.9604998  -1.9519119  -1.9373393  ... -1.4723451  -1.4607754
  -1.4521916 ]
 [-1.5644987  -1.5409694  -1.5463572  ... -1.3370459  -1.3214929
  -1.3119102 ]], shape=(494, 30), dtype=float32)

 took 0.01453089714050293  s with old method
tf.Tensor(
[[ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 [ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 [ 1.5503609   1.5152308   1.5787294  ...  0.9655902   0.94694483
   0.9274437 ]
 ...
 [-1.9604998  -1.9519119  -1.9373393  ... -1.4723451  -1.4607754
  -1.4521916 ]
 [-1.9604998  -1.9519119  -1.9373393  ... -1.4723451  -1.4607754
  -1.4521916 ]
 [

In [116]:
# vect = it.inside2
vect = tf.ragged.constant([[],[1,2,3,4],[5,4,3,2,1],[6],[99],[7,8,9,10,11,12,13]])
# print(tf.shape(vect)[0])
print("vect", vect)
c = tf.map_fn(sample, vect)
# print(c)

#wrong
# test = tf.gather(vect,tf.range(tf.shape(vect)[0]))[:3]
idx = tf.range(3)
print("\n idx", idx)
test = tf.gather(vect, idx , axis = 1)
print("\n test", test) #NOTE: indices with too few elements produce unexpected behavior
                        #that doesn't matter since they get suppressed anyways


vect <tf.RaggedTensor [[], [1, 2, 3, 4], [5, 4, 3, 2, 1], [6], [99], [7, 8, 9, 10, 11, 12, 13]]>

 idx tf.Tensor([0 1 2], shape=(3,), dtype=int32)

 test tf.Tensor(
[[ 1  2  3]
 [ 1  2  3]
 [ 5  4  3]
 [ 6 99  7]
 [99  7  8]
 [ 7  8  9]], shape=(6, 3), dtype=int32)
